CUNY-Specific Analytics
1. Which MTA bus routes are highly utilized by CUNY students?
2. How do violation rates compare among CUNY campuses?

In [1]:
import numpy as np
import pandas as pd

Data loading/cleaning - CUNY campuses

In [3]:
# url = https://data.ny.gov/Education/City-University-of-New-York-CUNY-University-Campus/i5b5-imzn/data
campuses = pd.read_csv("../raw_data/cuny_campuses.csv")
campuses.head()

,College or Institution Type,Campus,Campus Website,Address,City,State,Zip,Latitude,Longitude,Georeference
0,Community Colleges,Borough of Manhattan Community College,https://www.bmcc.cuny.edu/,199 Chambers Street,New York,NY,10007-1044,40.717367,-74.012178,POINT (-74.012178 40.717367)
1,Community Colleges,Bronx Community College,https://www.bcc.cuny.edu/,2155 University Avenue,Bronx,NY,10453-2804,40.856673,-73.910127,POINT (-73.910127 40.856673)
2,Community Colleges,Hostos Community College,http://hostos.cuny.edu,500 Grand Concourse,Bronx,NY,10451-5323,40.817828,-73.926862,POINT (-73.926862 40.817828)
3,Community Colleges,Kingsborough Community College,http://kbcc.cuny.edu,2001 Oriental Boulevard,Brooklyn,NY,11235-2333,40.578349,-73.934465,POINT (-73.934465 40.578349)
4,Community Colleges,LaGuardia Community College,https://www.laguardia.edu/,31-10 Thomson Avenue,Long Island City,NY,11101-3007,40.743951,-73.935154,POINT (-73.935154 40.743951)


Data loading/cleaning - Bus stops/routes

Data loading/cleaning - MTA Bus ACE Violations


Linking the datasets
- Which bus routes serve each CUNY campus
- Find the violation counts for those routes (+violation types?)
- Compare across CUNY campuses

Visualizations/Insights

Recommendations